<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Explainability%20models/IG_MuRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.7 MB/s eta 0:00:00


In [3]:
!pip install alibi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.7/522.7 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 10.5 MB/s eta 0:00:00


In [19]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import torch
from transformers import BertTokenizer
import matplotlib.pyplot as plt
from transformers import pipeline
from alibi.explainers.integrated_gradients import IntegratedGradients

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))

In [7]:
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)

In [22]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)
  return inputs

In [23]:
ig = IntegratedGradients(model.module)

AttributeError: ignored

In [16]:
text = "This is a sample input text."
X = adapter(text)
explanation = ig.explain(X, target=1)

TypeError: ignored

In [9]:
## Define Wrpper for making prediction using MuRIL model
class AutoModelWrapper(tf.keras.Model):

  def __init__(self, model_muril, **kwargs):
    super().__init__()
    self.model_muril = model_muril
  
  def call(self, inputs, attention_mask = None):
    out = self.model_muril(inputs,attention_mask = attention_mask)
    return tf.nn.softmax(out.logits)

  def get_config(self):
    return {}

  @classmethod
  def from_config(cls, config):
    return cls(**config)

auto_model = AutoModelWrapper(model)
max_features = 20000
max_len = 20

In [10]:
# Text sentence for the analysis
z_test_sample = ['I feel a little sad and angry these days']

# Tokenize the sentence and convert it into input tensor
z_test_sample = adapter(z_test_sample)
z_test_sample_input_ids = z_test_sample['input_ids']

# get tensor for model prediction
kwargs = {k: tf.constant(v) for k,v in z_test_sample.items() if k == 'attention_mask'}


In [11]:
## define the integrated gradient method
n_steps = 20
method = 'gausslegendre'
internal_batch_size = 5
ig = IntegratedGradients(auto_model, n_steps = n_steps, method = method, internal_batch_size = internal_batch_size) # TensorFlow model